In [1]:
REALTIME_ESTIMATOR = False
WEEK = 68

In [2]:
import os
import json
import warnings
import requests
import pandas as pd
from web3 import Web3
from urllib.request import urlopen
from google.cloud import bigquery
from google.cloud import bigquery_storage

In [3]:
# constants
week_1_start_ts = 1590969600
week_end_timestamp = week_1_start_ts + WEEK * 7 * 24 * 60 * 60
week_start_timestamp = week_end_timestamp - 7 * 24 * 60 * 60

BAL_addresses = {
    1: '0xba100000625a3754423978a60c9317c58a424e3d',
    137: '0x9a71012b13ca4d3d0cdc72a177df3ef03b0e76a3',
    42161: '0x040d1edc9569d4bab2d15287dc5a4f10f56a56b8'
}
networks = {
    1: 'ethereum',
    137: 'polygon',
    42161: 'arbitrum'
}

CLAIM_PRECISION = 12 # leave out of results addresses that mined less than CLAIM_THRESHOLD
CLAIM_THRESHOLD = 10**(-CLAIM_PRECISION)

reports_dir = f'reports/{WEEK}'
if not os.path.exists(reports_dir):
    os.mkdir(reports_dir)
def get_export_filename(network, token):
    return f'{reports_dir}/__{network}_{token}_subgraph.json'

In [4]:
if REALTIME_ESTIMATOR:
    warnings.warn('Running realtime estimator')
    
    from urllib.request import urlopen
    import json
    url = 'https://raw.githubusercontent.com/balancer-labs/bal-mining-scripts/master/reports/_current.json'
    jsonurl = urlopen(url)
    claims = json.loads(jsonurl.read())
    claimable_weeks = [20+int(w) for w in claims.keys()]
    most_recent_week = max(claimable_weeks)
    # delete the estimates for the most recent published week, since now there's an official value available on IPFS
    project_id = os.environ['GCP_PROJECT']
    sql = f'''
        DELETE FROM {project_id}.bal_mining_estimates.lp_estimates_multitoken
        WHERE week = {most_recent_week}
    '''
    client = bigquery.Client()
    query = client.query(sql)
    query.result()
    
    
    from datetime import datetime
    week_1_start = '01/06/2020 00:00:00 UTC'
    week_1_start = datetime.strptime(week_1_start, '%d/%m/%Y %H:%M:%S %Z')
    WEEK = int(1 + (datetime.utcnow() - week_1_start).days/7)  # this is what week we're actually in
    week_end_timestamp = week_1_start_ts + WEEK * 7 * 24 * 60 * 60
    week_start_timestamp = week_end_timestamp - 7 * 24 * 60 * 60
    week_end_timestamp = int(datetime.utcnow().timestamp())
    week_passed = (week_end_timestamp - week_start_timestamp)/(7*24*3600)

In [5]:
# get addresses that redirect
if REALTIME_ESTIMATOR:
    url = 'https://raw.githubusercontent.com/balancer-labs/bal-mining-scripts/master/config/redirect.json'
    jsonurl = urlopen(url)
    redirects = json.loads(jsonurl.read())
else:
    redirects = json.load(open('config/redirect.json'))

In [6]:
def get_start_block(network, start_timestamp):
    
    endpoint = {
        1: 'https://api.thegraph.com/subgraphs/name/blocklytics/ethereum-blocks',
        137: 'https://api.thegraph.com/subgraphs/name/matthewlilley/polygon-blocks',
        42161: 'https://api.thegraph.com/subgraphs/name/ianlapham/arbitrum-one-blocks'
    }
    
    query = '''
        {
          blocks(first: 1, orderBy: number, orderDirection: desc, where: { timestamp_lte: {}}) {
            number
          }
        }
    '''.replace('{','{{').replace('}','}}').replace('{{}}','{}').format(
        start_timestamp,
    )
    
    r = requests.post(endpoint[network], json = {'query': query})

    try:
        start_block = json.loads(r.content)['data']['blocks'][0]['number']
    except:
        raise Exception(json.loads(r.content)['errors'][0]['message'])
    
    return start_block

In [7]:
def get_pool_share_subgraph(pools_addresses, network, time_travel_block):
    
    endpoint = {
        1: 'https://api.thegraph.com/subgraphs/name/mendesfabio/balancer-analytics-v2',
        137: 'https://api.thegraph.com/subgraphs/name/mendesfabio/balancer-analytics-polygon-v2',
        42161: 'https://api.thegraph.com/subgraphs/name/mendesfabio/balancer-analytics-arbitrum-v2'
    }
    
    dfs = []
    
    for skip in range(0, 6000, 1000):
        query = '''
            {
              poolShares(
                first: 1000,
                skip: {},
                block: { number: {} },
                where: { pool_in: ["{}"], balance_gt: 0}) {
                  user {
                    id
                  }
                  pool {
                    id
                  }
                  balance
                }
            }
        '''.replace('{','{{').replace('}','}}').replace('{{}}','{}').format(
            skip,
            time_travel_block,
            '","'.join(pools_addresses)
        )
        
        r = requests.post(endpoint[network], json = {'query':query})
        
        try:
            p = json.loads(r.content)['data']['poolShares']
        except:
            raise Exception(json.loads(r.content)['errors'][0]['message'])
        
        df = pd.DataFrame(p)
        dfs.append(df)
                
        if len(df.index) < 1000:
            break        

    BPT_supply_df = pd.concat(dfs)
    BPT_supply_df['block'] = int(time_travel_block)
    BPT_supply_df['userAddress'] = BPT_supply_df['user'].map(lambda x: x['id'])
    BPT_supply_df['poolId'] = BPT_supply_df['pool'].map(lambda x: x['id'])
    BPT_supply_df['balance'] = BPT_supply_df['balance'].astype(float)
    BPT_supply_df['timestamp'] = int(week_start_timestamp)
    
    return BPT_supply_df

In [8]:
def get_bpt_tranfers_subgraph(pools_addresses, network, start_timestamp, end_timestamp):

    endpoint = {
        1: 'https://api.thegraph.com/subgraphs/name/mendesfabio/balancer-analytics-v2',
        137: 'https://api.thegraph.com/subgraphs/name/mendesfabio/balancer-analytics-polygon-v2',
        42161: 'https://api.thegraph.com/subgraphs/name/mendesfabio/balancer-analytics-arbitrum-v2'
    }
    
    dfs = []
    
    while True:

        query = '''
            {
              transfers(
                  first: 1000, 
                  orderBy: timestamp, 
                  where: { timestamp_gte: {}, timestamp_lt: {}, pool_in: ["{}"] }) {
                id
                pool {
                  id
                }
                fromAddress
                toAddress
                amount
                block
                timestamp
              } 
            }
        '''.replace('{','{{').replace('}','}}').replace('{{}}','{}').format(
            start_timestamp,
            end_timestamp,
            '","'.join(pools_addresses)
        )
        
        r = requests.post(endpoint[network], json = {'query':query})
        
        try:
            p = json.loads(r.content)['data']['transfers']
        except:
            raise Exception(json.loads(r.content)['errors'][0]['message'])

        df = pd.DataFrame(p)
        dfs.append(df)
        
        if len(df.index) < 1000:
            break
            
        start_timestamp = df['timestamp'].iloc[-1]

    BPT_supply_df = pd.concat(dfs)  
    BPT_supply_df['poolId'] = BPT_supply_df['pool'].map(lambda x: x['id'])
    BPT_supply_df['amount'] = BPT_supply_df['amount'].astype(float)
    BPT_supply_df['timestamp'] = BPT_supply_df['timestamp'].astype(int)
    BPT_supply_df['block'] = BPT_supply_df['block'].astype(int)
    BPT_supply_df = BPT_supply_df[['id', 'fromAddress', 'toAddress', 'poolId', 'block', 'timestamp', 'amount']]
    BPT_supply_df = BPT_supply_df.drop_duplicates()
    
    df1 = BPT_supply_df.copy()
    df1['userAddress'] = df1['fromAddress']
    df1['balance'] = - df1['amount']
    df1 = df1[['userAddress', 'poolId', 'block', 'timestamp', 'balance']]
    
    df2 = BPT_supply_df.copy()
    df2['userAddress'] = df2['toAddress']
    df2['balance'] = df2['amount']
    df2 = df2[['userAddress', 'poolId', 'block', 'timestamp', 'balance']]
    
    return df1.append(df2)

In [9]:
def transform_lm_data(shares, transfers):
    
    # merge and cumsum

    df_bpts = shares.append(transfers)

    df_bpts = df_bpts[df_bpts['userAddress'] != '0x0000000000000000000000000000000000000000']
    df_bpts = df_bpts[df_bpts['userAddress'] != '0xBA12222222228d8Ba445958a75a0704d566BF2C8']

    df_bpts = df_bpts.sort_values(by='block')

    df_bpts = df_bpts.groupby(['block', 'timestamp', 'userAddress', 'poolId']).sum().groupby(level=[2, 3]).cumsum().reset_index()

    # fill balance gaps

    blocks_pools = df_bpts[['block', 'timestamp', 'poolId']].drop_duplicates()
    lps_pools = df_bpts[['userAddress', 'poolId']].drop_duplicates()

    util = pd.merge(lps_pools, blocks_pools, on='poolId', how='inner')

    running_bpts = util.merge(df_bpts, on=['poolId', 'userAddress', 'block', 'timestamp'], how='left')

    running_bpts["balance"] = running_bpts.groupby(['userAddress', 'poolId'])['balance'].transform(lambda x: x.ffill().fillna(0))

    # calc timestamp diffs

    running_bpts = running_bpts.sort_values(['block'])
    running_bpts['duration'] = abs(running_bpts.groupby(['userAddress', 'poolId'])['timestamp'].diff(-1))

    running_bpts.loc[pd.isnull(running_bpts['duration']), 'duration'] = week_end_timestamp - running_bpts[pd.isnull(running_bpts['duration'])]['timestamp']

    # shares and integrator

    running_bpts['share'] = running_bpts['balance'] / running_bpts.groupby(['block', 'poolId'])['balance'].transform('sum')
    running_bpts['share_integral'] = running_bpts['share'] * running_bpts['duration']

    integrator = running_bpts.groupby(['poolId'])['share_integral'].sum().reset_index()
    integrator = integrator.rename(columns={'share_integral': 'integral'})

    # time-weighted shares

    tw_share = running_bpts.groupby(['userAddress', 'poolId'])['share_integral'].sum().reset_index()

    tw_share = tw_share.merge(integrator, on='poolId')

    tw_share['tw_share'] = tw_share['share_integral'] / tw_share['integral']

    tw_share = tw_share[['userAddress', 'poolId', 'tw_share']]
    tw_share = tw_share.rename(columns={'userAddress': 'miner', 'poolId': 'pool_address'})
    
    return tw_share

In [10]:
def v2_liquidity_mining(week, 
                        pools_addresses_and_tokens_earned,
                        BPT_share_df,
                        network):
    
    network_name = networks[network]
    
    BPT_share_df['miner'] = BPT_share_df['miner'].apply(Web3.toChecksumAddress)
    BPT_share_df.set_index(['pool_address','miner'], inplace=True)

    bal_mined_v2 = pools_addresses_and_tokens_earned.mul(BPT_share_df['tw_share'], axis=0)

    if REALTIME_ESTIMATOR:
        bal_mined_v2 *= week_passed

    miner_export = bal_mined_v2.groupby('miner').sum()

    for token in miner_export.columns:
        miner_export_v2 = miner_export[token].dropna()
        print(f'\n{miner_export_v2.sum()} {token} mined on {network_name}')

        v2_miners = pd.DataFrame(miner_export_v2).reset_index()
        n = len(v2_miners['miner'][v2_miners['miner'].isin(redirects.keys())])
        print(f'Redirect: {n} redirectors found')
        v2_miners['miner'] = v2_miners['miner'].apply(lambda x: redirects.get(x,x))
        miner_export_v2 = v2_miners.groupby('miner').sum()[token]

        if not REALTIME_ESTIMATOR:
            filename = get_export_filename(network_name, token)
            (
                miner_export_v2[miner_export_v2>=CLAIM_THRESHOLD]
                .apply(
                    lambda x: format(
                        x, 
                        f'.{CLAIM_PRECISION}f'
                    )
                )
                .to_json(filename, indent=4)
            )
    return miner_export

In [14]:
V2_LM_ALLOCATION_URL = 'https://raw.githubusercontent.com/balancer-labs/frontend-v2/master/src/lib/utils/liquidityMining/MultiTokenLiquidityMining.json'
jsonurl = urlopen(V2_LM_ALLOCATION_URL)

try:
    V2_ALLOCATION_THIS_WEEK = json.loads(jsonurl.read())[f'week_{WEEK}']
except KeyError:
    V2_ALLOCATION_THIS_WEEK = {}

full_export = pd.DataFrame()

for chain in V2_ALLOCATION_THIS_WEEK:
    print('------------------------------------------------------------------------------')
    print('\nChain: {}'.format(chain['chainId']))
    if chain['chainId'] in [1, 42161]:
        df = pd.DataFrame()
        for pool,rewards in chain['pools'].items():
            for r in rewards:
                pool_address = pool.lower()
                df.loc[pool_address,r['tokenAddress']] = r['amount']
        if len(df) == 0:
            print('No incentives for this chain')
            continue
        df.fillna(0, inplace=True)
        df.index.name = 'pool_address'
        bal_address = BAL_addresses[chain['chainId']]
        if bal_address in df.columns:
            bal_on_this_chain = df[bal_address].sum()
        else:
            bal_on_this_chain = 0
        print('BAL to be mined on this chain: {}'.format(bal_on_this_chain))
        
        start_block = get_start_block(chain['chainId'], week_start_timestamp)
        shares = get_pool_share_subgraph(df.index, chain['chainId'], start_block)
        transfers = get_bpt_tranfers_subgraph(df.index, chain['chainId'], week_start_timestamp, week_end_timestamp)
        lm_data = transform_lm_data(shares, transfers)
        
        chain_export = v2_liquidity_mining(WEEK, df, lm_data, chain['chainId'])
        chain_export['chain_id'] = chain['chainId']
        full_export = full_export.append(chain_export)
    else:
        pass

------------------------------------------------------------------------------

Chain: 1
BAL to be mined on this chain: 114000.0

101499.99999999936 0xba100000625a3754423978a60c9317c58a424e3d mined on ethereum
Redirect: 5 redirectors found

74999.99999999981 0x5a98fcbea516cf06857215779fd812ca3bef1b32 mined on ethereum
Redirect: 5 redirectors found

4700.0 0x81f8f0bb1cb2a06649e51913a151f0e7ef6fa321 mined on ethereum
Redirect: 5 redirectors found
------------------------------------------------------------------------------

Chain: 137
------------------------------------------------------------------------------

Chain: 42161
BAL to be mined on this chain: 6000.0

6000.000000000053 0x040d1edc9569d4bab2d15287dc5a4f10f56a56b8 mined on arbitrum
Redirect: 0 redirectors found


In [15]:
if not REALTIME_ESTIMATOR:
    mainnet_BAL = pd.read_json(
        get_export_filename(networks[1], BAL_addresses[1]), 
        typ='series', 
        convert_dates=False)

#     polygon_BAL = pd.read_json(
#         get_export_filename(networks[137], BAL_addresses[137]), 
#         typ='series', 
#         convert_dates=False)

    arbitrum_BAL = pd.read_json(
        get_export_filename(networks[42161], BAL_addresses[42161]), 
        typ='series', 
        convert_dates=False)

    mined_BAL = mainnet_BAL.add(arbitrum_BAL, fill_value=0) # TODO: add(arbitrum_BAL, fill_value=0)

    filename = '/_totalsLiquidityMining.json'
    (
        mined_BAL[mined_BAL>=CLAIM_THRESHOLD]
        .apply(
            lambda x: format(
                x, 
                f'.{CLAIM_PRECISION}f'
            )
        )
        .to_json(reports_dir+filename, indent=4)
    )
    print('Total BAL mined: {}'.format(mined_BAL.sum()))

Total BAL mined: 107499.99999999942


In [16]:
full_export_bkp = full_export.copy()

In [17]:
full_export = (
    full_export_bkp
    .set_index('chain_id', append=True)
    .melt(
        var_name = 'token_address', 
        value_name = 'earned',
        ignore_index=False)
    .reset_index()
)
full_export.rename(columns={'miner':'address'}, inplace=True)
full_export.set_index(['address','chain_id','token_address'], inplace=True)
full_export.dropna(inplace=True)
full_export['earned'] = full_export['earned'].apply(lambda x: format(x, f'.{18}f'))

# Update real time estimates in GBQ

In [18]:

if REALTIME_ESTIMATOR:
    # zero previous week's velocity
    sql = f'''
        UPDATE {project_id}.bal_mining_estimates.lp_estimates_multitoken
        SET velocity = '0'
        WHERE week = {WEEK-1}
    '''
    client = bigquery.Client()
    query = client.query(sql)
    query.result();
    
    try:
        sql = f'select * from bal_mining_estimates.lp_estimates_multitoken WHERE week = {WEEK}'
        prev_estimate = pd.read_gbq(sql, 
                        project_id=os.environ['GCP_PROJECT'])
        prev_estimate.set_index(['address','chain_id','token_address'], inplace=True)
        prev_estimate_timestamp = prev_estimate.iloc[0]['timestamp']
    except:
        prev_estimate_timestamp = 0
    if prev_estimate_timestamp < week_start_timestamp:
        #previous estimate is last week's; compute velocity between from week_start_timestamp and week_end_timestamp
        delta_t = (week_end_timestamp - week_start_timestamp)
        earned = full_export['earned'].astype(float)
        full_export['velocity'] = (earned/delta_t).apply(lambda x: format(x, f'.{18}f'))
    else:
        #compute velocity based on increase and time passed
        delta_t = (week_end_timestamp - prev_estimate_timestamp)
        diff_estimate = full_export.join(prev_estimate, rsuffix='_prev').fillna(0)
        cur_earned = diff_estimate['earned'].astype(float)
        prev_earned = diff_estimate['earned_prev'].astype(float)
        full_export['velocity'] = ((cur_earned-prev_earned)/delta_t).apply(lambda x: format(x, f'.{18}f'))

    full_export['timestamp'] = week_end_timestamp
    full_export['week'] = WEEK
    full_export.reset_index(inplace=True)
    full_export.to_gbq('bal_mining_estimates.lp_estimates_multitoken_staging', 
                       project_id=os.environ['GCP_PROJECT'], 
                       if_exists='replace')

    # merge staging into prod
    sql = '''
    MERGE bal_mining_estimates.lp_estimates_multitoken prod
    USING bal_mining_estimates.lp_estimates_multitoken_staging stage
    ON prod.address = stage.address
    AND prod.week = stage.week
    AND prod.chain_id = stage.chain_id
    AND prod.token_address = stage.token_address
    WHEN MATCHED THEN
        UPDATE SET 
            earned = stage.earned,
            velocity = stage.velocity,
            timestamp = stage.timestamp
    WHEN NOT MATCHED BY TARGET THEN
        INSERT (address, week, chain_id, token_address, earned, velocity, timestamp)
        VALUES (address, week, chain_id, token_address, earned, velocity, timestamp)
    '''
    client = bigquery.Client()
    query = client.query(sql)
    query.result()

In [ ]:
from src.bal4gas_V1 import compute_bal_for_gas as compute_bal_for_gas_V1
from src.bal4gas_V2 import compute_bal_for_gas as compute_bal_for_gas_V2

if not REALTIME_ESTIMATOR:
    allowlist = pd.read_json(
        f'https://raw.githubusercontent.com/balancer-labs/assets/master/generated/bal-for-gas.json', 
        orient='index').loc['homestead'].values
    gas_allowlist = pd.Series(allowlist).str.lower().tolist()
    gas_allowlist.append('0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee')

    
    v1 = compute_bal_for_gas_V1(week_start_timestamp, week_end_timestamp, gas_allowlist, plot=True, verbose=True)

    gas_allowlist.remove('0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee')
    gas_allowlist.append('0x0000000000000000000000000000000000000000')
    v2 = compute_bal_for_gas_V2(week_start_timestamp, week_end_timestamp, gas_allowlist, plot=True, verbose=True)
    
    merge = v1.append(v2)

    totals_bal4gas = merge[['address','bal_reimbursement']].groupby('address').sum()['bal_reimbursement']
    totals_bal4gas[totals_bal4gas>=CLAIM_THRESHOLD].apply(       lambda x: format(x, f'.{CLAIM_PRECISION}f')).to_json(reports_dir+'/_gasReimbursement.json',
       indent=4)

    # combine BAL from liquidity mining and gas reimbursements
    totals = mainnet_BAL.add(totals_bal4gas, fill_value=0)
    totals[totals>=CLAIM_THRESHOLD].apply(       lambda x: format(x, f'.{CLAIM_PRECISION}f')).to_json(reports_dir+'/_totals.json',
       indent=4)

In [ ]:
if not REALTIME_ESTIMATOR:
    print('Final Check Totals BAL')
    
    
    _ethereum = pd.read_json(
        get_export_filename(networks[1], BAL_addresses[1]), 
        typ='series', 
        convert_dates=False).sum()

#     _polygon = pd.read_json(
#         get_export_filename(networks[137], BAL_addresses[137]), 
#         typ='series', 
#         convert_dates=False).sum()
    
    _arbitrum = pd.read_json(
        get_export_filename(networks[42161], BAL_addresses[42161]), 
        typ='series', 
        convert_dates=False).sum()
    
    
    _lm_all_networks = pd.read_json(reports_dir+'/_totalsLiquidityMining.json', orient='index').sum().values[0]
    _claim = pd.read_json(reports_dir+'/_totals.json', orient='index').sum().values[0]
    print(f'Liquidity Mining Ethereum: {format(_ethereum, f".{CLAIM_PRECISION}f")}')
#     print(f'Liquidity Mining Polygon: {format(_polygon, f".{CLAIM_PRECISION}f")}')
    print(f'Liquidity Mining Arbitrum: {format(_arbitrum, f".{CLAIM_PRECISION}f")}')
    print(f'Liquidity Mining All Networks: {format(_lm_all_networks, f".{CLAIM_PRECISION}f")}')
    print(f'Gas Reimbursement week {WEEK}: {format(_claim-_ethereum, f".{CLAIM_PRECISION}f")}')
    print(f'Claims: {format(_claim, f".{CLAIM_PRECISION}f")}')
    
    # check all reports files
    print('\nReports totals:')
    checks = {}
    for f in os.listdir(reports_dir):
        _sum = pd.read_json(reports_dir+'/'+f, orient='index').sum().values[0]
        checks[f] = _sum
    display(pd.DataFrame.from_dict(checks, orient='index', columns=['total']).sort_index())

# Comparisons

In [ ]:
with open(f'reports/{WEEK}/__arbitrum_0x040d1edc9569d4bab2d15287dc5a4f10f56a56b8_subgraph.json', 'r') as f:
    data = json.load(f)
    
subgraph = pd.DataFrame({'subgraph': data})

with open(f'reports/{WEEK}/__arbitrum_0x040d1edc9569d4bab2d15287dc5a4f10f56a56b8.json', 'r') as f:
    data = json.load(f)
    
bigquery = pd.DataFrame({'bigquery': data})

results = subgraph.join(bigquery)

results['subgraph'] = results['subgraph'].astype(float)
results['bigquery'] = results['bigquery'].astype(float)

results['diff'] = abs(results['bigquery'] - results['subgraph'])

results.sort_values(by='diff')

In [ ]:
with open(f'reports/{WEEK}/__ethereum_0xba100000625a3754423978a60c9317c58a424e3d_subgraph.json', 'r') as f:
    data = json.load(f)
    
subgraph = pd.DataFrame({'subgraph': data})

with open(f'reports/{WEEK}/__ethereum_0xba100000625a3754423978a60c9317c58a424e3d.json', 'r') as f:
    data = json.load(f)
    
bigquery = pd.DataFrame({'bigquery': data})

results = subgraph.join(bigquery)

results['subgraph'] = results['subgraph'].astype(float)
results['bigquery'] = results['bigquery'].astype(float)

results['diff'] = abs(results['bigquery'] - results['subgraph'])

results[results['diff'] > 0].sort_values('diff')